# Pinecone Canopy library quick start notebook

**Canopy** is a Software Development Kit (SDK) for AI applications. Canopy allows you to test, build and package Retrieval Augmented Applications with Pinecone Vector Database. 

This notebook introduces the quick start steps for working with Canopy library. You can find more details about this project and advanced use in the project [documentation](../README.md).


## Prerequisites

install canopy library

In [33]:
!pip install -qU canopy-sdk


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


By default, Canopy uses Pinecone and OpenAI so we need to configure the related API keys.

To get Pinecone free trial API key and environment register or log into your Pinecone account in the [console](https://app.pinecone.io/). You can access your API key from the "API Keys" section in the sidebar of your dashboard, and find the environment name next to it.

You can find your free trial OpenAI API key [here](https://platform.openai.com/account/api-keys). You might need to login or register to OpenAI services.



In [34]:
import os

os.environ["PINECONE_API_KEY"] = os.environ.get('PINECONE_API_KEY') or 'YOUR_PINECONE_API_KEY'
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'

## Pinecone Documentation Dataset

Now we'll load a crawl from 25/10/23 of pinecone docs [website](https://docs.pinecone.io/docs/).

We will use this data to demonstrate how to build a RAG pipeline to answer questions about Pinecone DB.

In [36]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


Each record in this dataset represents a single page in Pinecone's documentation. Each row contains a unique id, the raw text of the page in markdown language, the url of the page as "source" and some metadata. 

## Init a Tokenizer


Many of Canopy's components are using tokenization, which is a process that splits text into tokens - basic units of text (like word or sub-words) that are used for processing. Therefore, Canopy uses a singleton `Tokenizer` object which needs to be initialized once.

In [37]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

After initializing the global object, we can simply create an instance from anywhere in our code, without providing any parameters:

In [38]:
from canopy.tokenizer import Tokenizer

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

['Hello', ' world', '!']

## Creating a KnowledgBase to store our data for search

The `KnowledgeBase` object is responsible for storing and indexing textual documents.

Once documents are indexed, the `KnowledgeBase` can be queried with a new unseen text passage, for which the most relevant document chunks are retrieved.

The `KnowledgeBase` holds a connection to a Pinecone index and provides a simple API to insert, delete and search textual documents.

The `KnowledgeBase`'s `upsert()` operation is used to index new documents, or update already stored documents. The `upsert` process splits each document's text into smaller chunks, transforms these chunks to vector embeddings, then upserts those vectors to the underlying Pinecone index. At Query time, the `KnowledgeBase` transforms the textual query text to a vector in a similar manner, then queries the underlying Pinecone index to retrieve the top-k most closely matched document chunks.

Here we create a `KnowledgeBase` with our desired index name: 

In [39]:
from canopy.knowledge_base import KnowledgeBase

INDEX_NAME = "my-index"

kb = KnowledgeBase(index_name=INDEX_NAME)

In the first one-time setup of a new Canopy service, an underlying Pinecone index needs to be created. If you have created a Canopy-enabled Pinecone index before - you can skip this step.

Note: Since Canopy uses a dedicated data schema, it is not recommended to use a pre-existing Pinecone index that wasn't created by Canopy's `create_canopy_index()` method.

In [40]:
from canopy.knowledge_base import list_canopy_indexes
if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index()

You can see the index created in Pinecone's [console](https://app.pinecone.io/)

next time we would like to init a knowledge base instance to this index, we can simply call the connect method:

In [41]:
kb = KnowledgeBase(index_name=INDEX_NAME)
kb.connect()

> 💡 Note: a knowledge base must be connected to an index before executing any operation. You should call `kb.connect()` to connect  an existing index or call `kb.create_canopy_index(INDEX_NANE)` before calling any other method of the KB 

## Upsert data to our KnowledgBase

First, we need to convert our dataset to list of `Document` objects

Each document object can hold id, text, source and metadata:

In [42]:
from canopy.models.data_models import Document

example_docs = [Document(id="1",
                      text="This is text for example",
                      source="https://url.com"),
                Document(id="2",
                        text="this is another text",
                        source="https://another-url.com",
                        metadata={"my-key": "my-value"})]

The data in our example dataset is already provided in this schema, so we can simply iterate over it and instantiate `Document` objects:

In [43]:
documents = [Document(**row) for _, row in data.iterrows()]

Now we are ready to upsert our data, with only a single command:

In [44]:
from tqdm.auto import tqdm

batch_size = 10

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/6 [00:00<?, ?it/s]

Internally, the KnowledgeBase handles all the processing needed to Index the documents. Each document's text is chunked to smaller pieces and encoded to vector embeddings that can be then upserted directly to Pinecone. Later in this notebook we'll learn how to tune and customize this process.

## Query the KnowledgeBase

Now we can query the knowledge base. The KnowledgeBase will use its default parameters like `top_k` to execute the query:

In [45]:
def print_query_results(results):
    for query_results in results:
        print('query: ' + query_results.query + '\n')
        for document in query_results.documents:
            print('document: ' + document.text.replace("\n", "\\n"))
            print("title: " + document.metadata["title"])
            print('source: ' + document.source)
            print(f"score: {document.score}\n")

In [46]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity")])

print_query_results(results)

query: p1 pod capacity

document: ### s1 pods\n\n\nThese storage-optimized pods provide large storage capacity and lower overall costs with slightly higher query latencies than p1 pods. They are ideal for very large indexes with moderate or relaxed latency requirements.\n\n\nEach s1 pod has enough capacity for around 5M vectors of 768 dimensions.\n\n\n### p1 pods\n\n\nThese performance-optimized pods provide very low query latencies, but hold fewer vectors per pod than s1 pods. They are ideal for applications with low latency requirements (<100ms).\n\n\nEach p1 pod has enough capacity for around 1M vectors of 768 dimensions.
title: indexes
source: https://docs.pinecone.io/docs/indexes
score: 0.844001234

document: ## Pod storage capacity\n\n\nEach **p1** pod has enough capacity for 1M vectors with 768 dimensions.\n\n\nEach **s1** pod has enough capacity for 5M vectors with 768 dimensions.\n\n\n## Metadata\n\n\nMax metadata size per vector is 40 KB.\n\n\nNull metadata values are not sup

You can change the `top_k` parameter, to determine the number of top query results that will be returned and also to provide a [metadata filter](https://docs.pinecone.io/docs/metadata-filtering).

In [47]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity",
                          metadata_filter={"source": "https://docs.pinecone.io/docs/limits"},
                          top_k=2)])

print_query_results(results)

query: p1 pod capacity

document: ## Pod storage capacity\n\n\nEach **p1** pod has enough capacity for 1M vectors with 768 dimensions.\n\n\nEach **s1** pod has enough capacity for 5M vectors with 768 dimensions.\n\n\n## Metadata\n\n\nMax metadata size per vector is 40 KB.\n\n\nNull metadata values are not supported. Instead of setting a key to hold a null value, we recommend you remove that key from the metadata payload.\n\n\nMetadata with high cardinality, such as a unique value for every vector in a large index, uses more memory than expected and can cause the pods to become full.
title: limits
source: https://docs.pinecone.io/docs/limits
score: 0.842464507

document: ## Retention\n\n\nIn general, indexes on the Starter (free) plan are archived as collections and deleted after 7 days of inactivity; for indexes created by certain open source projects such as AutoGPT, indexes are archived and deleted after 1 day of inactivity. To prevent this, you can send any API request to Pinecone a

As you can see above, using the metadata filter we get results only from the "limits" page

## Query the Context Engine

`ContextEngine` is an object responsible for retrieving the most relevant context for a given query and token budget.  

While `KnowledgeBase` retrieves the full `top-k` structured documents for each query including all the metadata related to them, the context engine in charge of transforming this information to a "prompt ready" context that can later feeded to an LLM. To achieve this the context engine holds a `ContextBuilder` object that takes query results from the knowledge base and returns a `Context` object. The `ContextEngine`'s default behavior is to use a `StuffingContextBuilder`, which simply stacks retrieved document chunks in a JSON-like manner, hard limiting by the number of chunks that fit the `max_context_tokens` budget. More complex behaviors can be achieved by providing a custom `ContextBuilder` class.

In [48]:
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [49]:
import json

result = context_engine.query([Query(text="capacity of p1 pods", top_k=5)], max_context_tokens=512)

print(result.to_text(indent=2))
print(f"\n# tokens in context returned: {result.num_tokens}")

{
  "query": "capacity of p1 pods",
  "snippets": [
    {
      "source": "https://docs.pinecone.io/docs/indexes",
      "text": "### s1 pods\n\n\nThese storage-optimized pods provide large storage capacity and lower overall costs with slightly higher query latencies than p1 pods. They are ideal for very large indexes with moderate or relaxed latency requirements.\n\n\nEach s1 pod has enough capacity for around 5M vectors of 768 dimensions.\n\n\n### p1 pods\n\n\nThese performance-optimized pods provide very low query latencies, but hold fewer vectors per pod than s1 pods. They are ideal for applications with low latency requirements (<100ms).\n\n\nEach p1 pod has enough capacity for around 1M vectors of 768 dimensions."
    },
    {
      "source": "https://docs.pinecone.io/docs/indexes",
      "text": "### p2 pods\n\n\nThe p2 pod type provides greater query throughput with lower latency. For vectors with fewer than 128 dimension and queries where `topK` is less than 50, p2 pods suppor

As you can see above, although we set `top_k=5`, context engine retreived only 3 results in order to satisfy the 512 tokens limit. Also, the documents in the context contain only the text and source and not all the metadata that is not necessarily needed by the LLM. 

## Knowledgeable chat engine

Now we are ready to start chatting with our data!

Canopy's `ChatEngine` is a one-stop-shop RAG-infused Chatbot. The `ChatEngine` wraps an underlying LLM such as OpenAI's ChatGPT, enhancing it by providing relevant context from the user's knowledge base. It also automatically phrases search queries out of the chat history and send them to the knowledge base.

In [50]:
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

In [51]:
from typing import Tuple
from canopy.models.data_models import Messages, UserMessage, AssistantMessage

def chat(new_message: str, history: Messages) -> Tuple[str, Messages]:
    messages = history + [UserMessage(content=new_message)]
    response = chat_engine.chat(messages)
    assistant_response = response.choices[0].message.content
    return assistant_response, messages + [AssistantMessage(content=assistant_response)]

In [52]:
from IPython.display import display, Markdown

history = []
response, history = chat("What is the capacity of p1 pods?", history)
display(Markdown(response))

The capacity of p1 pods is enough for around 1 million vectors of 768 dimensions. Source: [Pinecone Documentation](https://docs.pinecone.io/docs/indexes)

In [53]:
response, history = chat("And for what latency requirements does it fit?", history)
display(Markdown(response))

P1 pods are ideal for applications with low latency requirements, specifically those that require latencies of less than 100 milliseconds. Source: [Pinecone Documentation](https://docs.pinecone.io/docs/indexes)

> 💡 Note: Canopy calls the underlying LLM, providing both the user-provided chat history and a generated `Context` prompt. This might surpass the `ChatEngine`'s configured `max_prompt_tokens`. By default, the `ChatEngine` would truncate the oldest messages in the chat history to avoid exceeding this limit. This behavior in configurable, as explained in the [documentation](https://github.com/pinecone-io/canopy/blob/main/src/canopy/chat_engine/chat_engine.py)

## Customization Example

Canopy built as a modular library, where each component can fully be customized by the user.

Before we start, we would like to have a quick overview of the inner components used by the knowledge base:

- **Index**: A Pinecone index that holds the vector representations of the documents.
- **Chunker**: A `Chunker` object that is used to chunk the documents into smaller pieces of text.
- **Encoder**: An `RecordEncoder` object that is used to encode the chunks and queries into vector representations.

In the following example, we show how you can customize the `Chunker` component used by the knowledge base.

First, we will create a dummy chunker class that simply chunks the text by new lines `\n`.

In [54]:
from typing import List
from canopy.knowledge_base.chunker.base import Chunker
from canopy.knowledge_base.models import KBDocChunk

class NewLineChunker(Chunker):

     def chunk_single_document(self, document: Document) -> List[KBDocChunk]:
        line_chunks = [chunk
                       for chunk in document.text.split("\n")]
        return [KBDocChunk(id=self.generate_chunk_id(document.id, i),
                           document_id=document.id,
                           text=text_chunk,
                           source=document.source,
                           metadata=document.metadata)
                for i, text_chunk in enumerate(line_chunks)]
    
     async def achunk_single_document(self, document: Document) -> List[KBDocChunk]:
        raise NotImplementedError()

In [55]:
chunker = NewLineChunker()

document = Document(id="id1",
                    text="This is first line\nThis is the second line",
                    source="example",
                    metadata={"title": "newline"})
chunker.chunk_single_document(document)

[KBDocChunk(id='id1_0', text='This is first line', source='example', metadata={'title': 'newline'}, document_id='id1'),
 KBDocChunk(id='id1_1', text='This is the second line', source='example', metadata={'title': 'newline'}, document_id='id1')]

Now we can initialize a new knowledge base to use our new chunker:

In [56]:
kb = KnowledgeBase(index_name=INDEX_NAME,
                   chunker=chunker)
kb.connect()

And upsert our example document:

In [57]:
kb.upsert([document])

In [58]:
results = kb.query([Query(text="second line",
                          metadata_filter={"title": "newline"})])

print_query_results(results)

query: second line

document: This is the second line
title: newline
source: example
score: 0.928711653

document: This is first line
title: newline
source: example
score: 0.887627542


As we can see above, our knowledge base split the document by a new line as expected.

Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [59]:
kb.delete_index()